In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torchvision import transforms
import torchvision.transforms.functional as F
import random
from PIL import Image
from pathlib import Path

In [2]:
file ='AI_Powered_Eduction_system'
def root(root_name:str=file):
    for parent in Path().resolve().parents:
        if parent.name == root_name:
            return(parent) 
ROOT = root()

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce GTX 1650


In [4]:
train_df=pd.read_csv(ROOT/r'data\Train_datasets\Digit_Recognizer\mnist_train.csv')
test_df = pd.read_csv(ROOT/r'data\Train_datasets\Digit_Recognizer\mnist_test.csv')

In [5]:
train_df.shape

(60000, 785)

In [6]:
test_df.shape

(10000, 785)

In [7]:
train_df.head(10)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_df.describe()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,60000.000000,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,4.453933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,2.889270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


In [9]:
r,g,b = [random.uniform(0.5,1.5) for i in range(3)]
print(r,g,b)

0.8939536771000054 0.6481260951937052 0.6069185168999922


In [10]:
class Colourize:
    def __call__(self,img):
        img = F.to_pil_image(img) if isinstance(img,torch.Tensor) else img
        img = img.convert('RGB')
        r,g,b = [random.uniform(0.5,1.5) for i in range(3)]
        img = F.adjust_contrast(img,2)
        img = F.to_tensor(img)
        img[0] *= r
        img[1] *= g
        img[2] *= b
        img = torch.clamp(img,0,1)
        return img

In [11]:
transform = transforms.Compose([transforms.Resize((28,28)),Colourize(),transforms.RandomAutocontrast(p=0.5),transforms.Normalize((0.5,),(0.5,))])

In [12]:
class ImageDataset(Dataset):
    def __init__(self,df,transform=None):
        self.labels = df.iloc[:,0].values.astype(np.int64)
        self.images = df.iloc[:,1:].values.astype(np.float32)
        self.transform = transform

    def __len__(self):
        return len(self.labels)
    def __getitem__(self,idx):
        img = self.images[idx].reshape(28,28)
        img = np.expand_dims(img,axis=0)
        img = torch.tensor(img)
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.labels[idx],dtype=torch.long)
        return img,label

In [13]:
train_data = ImageDataset(train_df,transform=transform)

In [14]:
train_loader = DataLoader(train_data,batch_size=16,shuffle=True)

In [15]:
image,label = next(iter(train_loader))
print(image.shape)

torch.Size([16, 3, 28, 28])


In [16]:
class Net(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        self.Feature_extractor = nn.Sequential(
            nn.Conv2d(3,32,kernel_size=3,padding=1,stride=1),
            nn.ELU(),
            nn.Conv2d(32,64,kernel_size=3,padding=1,stride=1),
            nn.ELU(),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(nn.Linear(64*28*28,128),nn.Linear(128,num_classes))
    def forward(self,x):
        x = self.Feature_extractor(x)
        x = self.classifier(x)
        return x

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
num = 10
net = Net(num_classes=num)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.001)
net.train(True)
for epoch in range(30):
    for image,label in train_loader:
        image,label=image.to(device),label.to(device)
        optimizer.zero_grad()
        outputs = net(image)
        loss = criterion(outputs,label)
        loss.backward()
        optimizer.step()
    print(f'epoch:{epoch}')


epoch:0
epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
epoch:11
epoch:12
epoch:13
epoch:14
epoch:15
epoch:16
epoch:17
epoch:18
epoch:19
epoch:20
epoch:21
epoch:22
epoch:23
epoch:24
epoch:25
epoch:26
epoch:27
epoch:28
epoch:29


In [19]:
from torchmetrics import Precision,Recall

In [20]:
test_df.head(10)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
test_df.describe()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,10000.000000,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.000000,10000.000000,10000.000000,10000.0000,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
mean,4.443400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179300,0.163600,0.052600,0.0006,0.0,0.0,0.0,0.0,0.0,0.0
std,2.895865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.674149,5.736072,2.420004,0.0600,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,253.000000,156.000000,6.0000,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
test_transform =transforms.Compose([transforms.Resize((28,28)),transforms.Normalize((0.5,),(0.5,))])

In [23]:
test_data = ImageDataset(test_df,transform=test_transform)
test_loader = DataLoader(test_data,batch_size=8,shuffle=True)

In [24]:
image,label = next(iter(test_loader))
image = image.repeat(1,3,1,1)
print(image.shape)

torch.Size([8, 3, 28, 28])


In [25]:
met_prec = Precision(task='multiclass',num_classes=num,average='macro')
met_prec.to(device)
met_rec = Recall(task='multiclass',num_classes=num,average='macro')
met_rec.to(device)
net.eval()
with torch.no_grad():
    for image,label in test_loader:
        image,label = image.to(device),label.to(device)
        image = image.repeat(1,3,1,1)
        outputs = net(image)
        _,preds =torch.max(outputs,1)
        met_prec(preds,label)
        met_rec(preds,label)
precision = met_prec.compute()
recall = met_rec.compute()
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Precision: 0.9201034307479858
Recall: 0.9077959656715393


In [26]:
trans = transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor(),transforms.Normalize((0.5),(0.5))])

In [27]:
image = Image.open(ROOT/r'data\test_data\numbers\3-Figure2-1.png').convert('RGB')
image = image.resize((28,28),resample=Image.Resampling.LANCZOS)
image = trans(image).unsqueeze(0)
with torch.no_grad():
    image = image.to(device)
    net.to(device)
    op = net(image)
    _,pred =torch.max(op,1)
    print(op)
    print(int(pred[0]))

tensor([[ -79.4523, -113.6020,  -52.9934, -116.3173,  281.9705,   86.5527,
           15.2725,   72.8913,    8.8624,   92.0449]], device='cuda:0')
4


In [29]:
image = Image.open(ROOT/r'data\test_data\numbers\seven.jpg').convert('RGB')
image = image.resize((28,28),resample=Image.Resampling.LANCZOS)
image = trans(image).unsqueeze(0)
with torch.no_grad():
    image = image.to(device)
    net.to(device)
    op = net(image)
    _,pred =torch.max(op,1)
    print(op)
    print(int(pred[0]))

tensor([[  20.4479,   61.5779,  279.2481,   83.5138, -184.9701,  -92.1989,
          -56.5547,  207.7322,   36.5277, -197.2939]], device='cuda:0')
2


In [30]:
torch.save(net.state_dict(),ROOT/r'Models\digit_model_weights.pth')